In [3]:
import json
train_file = open("dataset/implicit_train.json",)
test_file = open("dataset/implicit_test.json",)
dev_file = open("dataset/implicit_dev.json",)
train_data = json.load(train_file)
test_data = json.load(test_file)
dev_data = json.load(dev_file)
print(len(train_data),len(test_data),len(dev_data))


13046 1046 1183


In [5]:
import pandas as pd
df = pd.read_json (r'dataset/implicit_train.json')
df.to_csv (r'dataset/implicit_train.csv', index = None)

In [10]:
!git lfs install

'git' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [11]:
!git lfs clone https://huggingface.co/roberta-base

'git' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [19]:
keyword_dict = {0:[],1:[],2:[],3:[]}
def get_keyword(data):
    for dict in data:
        for index in dict["label"]:
            tmp_ls = []
            if not dict["conn"] in keyword_dict[index]:
                tmp_ls = keyword_dict[index]
                tmp_ls.append(dict["conn"])
get_keyword(train_data)
for k,v in keyword_dict.items():
    print(k,":",v,"\n")

'''
0 : comparison
1 : contingent
2 : expansion
3 : temporal
'''
            

0 : ['however', 'while', 'yet', 'but', 'nevertheless', 'on the other hand', 'in fact', 'although', 'by comparison', 'whereas', 'by contrast', 'and', 'even though', 'still', 'though', 'rather', 'instead', 'on the contrary', 'in contrast', 'in turn', 'meanwhile', 'so', 'in comparison', 'finally', 'in particular', 'at the same time', 'because', 'then', 'nonetheless'] 

1 : ['because', 'as a result', 'consequently', 'inasmuch as', 'thus', 'therefore', 'so', 'since', 'as', 'accordingly', 'but', 'furthermore', 'then', 'in turn', 'indeed', 'and', 'in fact', 'for one thing', 'specifically', 'as it turns out', 'insofar as', 'hence', 'so that', 'ever since', 'in other words', 'that is', 'to this end', 'rather', 'as a consequence', 'besides', 'in particular', 'for example', 'for instance', 'in the end', 'subsequently', 'after', 'also', 'ultimately', 'finally', 'for', 'in short', 'plus'] 

2 : ['and', 'for example', 'although', 'in short', 'rather', 'specifically', 'also', 'in particular', 'in sum

In [2]:
import torch
import torch.utils.data as torchdata

class Dataset(torchdata.Dataset):
    def __init__(self, data_path):
        super(Dataset, self).__init__()
        self.d1, self.d2, self.d3, self.d4 = torch.load(data_path)

    def __getitem__(self, index):
        return self.d1[index], self.d2[index], self.d3[index], self.d4[index]

    def __len__(self):
        return len(self.d4)

class Data(object):
    def __init__(self, use_cuda, conf, batch_size=None):
        if batch_size is None:
            batch_size = conf.batch_size
        kwargs = {'batch_size':batch_size, 'shuffle':conf.shuffle, 'drop_last':False}
        if use_cuda:
            kwargs['pin_memory'] = True
        if conf.corpus_splitting == 1:
            pre = './data/processed/lin/'
        elif conf.corpus_splitting == 2:
            pre = './data/processed/ji/'
        elif conf.corpus_splitting == 3:
            pre = './data/processed/l/'
        train_data = Dataset(pre+'train.pkl')
        dev_data = Dataset(pre+'dev.pkl')
        test_data = Dataset(pre+'test.pkl')
        self.train_size = len(train_data)
        self.dev_size = len(dev_data)
        self.test_size = len(test_data)
        self.train_loader = torchdata.DataLoader(train_data, **kwargs)
        self.dev_loader = torchdata.DataLoader(dev_data, **kwargs)
        self.test_loader = torchdata.DataLoader(test_data, **kwargs)

In [3]:
import torch
from datetime import datetime

class Config(object):
    def __init__(self, classnum=11, splitting=2):
        self.i2sense = [
            'Temporal.Asynchronous', 'Temporal.Synchrony', 'Contingency.Cause',
            'Contingency.Pragmatic cause', 'Comparison.Contrast', 'Comparison.Concession',
            'Expansion.Conjunction', 'Expansion.Instantiation', 'Expansion.Restatement',
            'Expansion.Alternative','Expansion.List'
        ]
        self.sense2i = {
            'Temporal.Asynchronous':0, 'Temporal.Synchrony':1, 'Contingency.Cause':2,
            'Contingency.Pragmatic cause':3, 'Comparison.Contrast':4, 'Comparison.Concession':5,
            'Expansion.Conjunction':6, 'Expansion.Instantiation':7, 'Expansion.Restatement':8,
            'Expansion.Alternative':9,'Expansion.List':10
        }
        self.i2senseclass = ['Temporal', 'Contingency', 'Comparison', 'Expansion']
        self.senseclass2i = {'Temporal':0, 'Contingency':1, 'Comparison':2, 'Expansion':3}

        self.four_or_eleven = classnum         # 11, 4, 2
        self.corpus_splitting = splitting        # 1 for Lin, 2 for Ji, 3 for 4-way and binary
        if self.four_or_eleven == 4 or self.four_or_eleven == 2:
            self.corpus_splitting = 3
        self.binclass = 0         # 0, 1, 2, 3 self.senseclass2i

        self.wordvec_path = '~/Projects/GoogleNews-vectors-negative300.bin.gz'
        self.wordvec_dim = 300
        self.max_sent_len = 100

        ################################################################################
        # attention
        self.attn_topk = 2
        self.attn_dropout = 0
        
        ###############################################################################
        # char/sub
        self.need_char = False
        self.need_sub = True
        if self.need_sub:
            self.need_char = False
        self.char_num = 262
        self.char_padding_idx = 261
        if self.need_sub:
            if self.corpus_splitting == 1:
                self.char_num = 982
            elif self.corpus_splitting == 2:
                self.char_num = 982
            elif self.corpus_splitting == 3:
                self.char_num = 982
            self.char_padding_idx = 0
        self.char_embed_dim = 50
        self.char_enc_dim = 50
        self.char_filter_num = 2
        self.char_filter_dim = [2, 3]
        self.char_dropout = 0
        self.char_hid_dim = self.char_enc_dim * self.char_filter_num

        ###############################################################################
        # elmo
        self.need_elmo = True
        self.elmo_options = '~/Projects/ELMo/elmo_2x4096_512_2048cnn_2xhighway_options.json'
        self.elmo_weights = '~/Projects/ELMo/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5'
        self.elmo_dropout = 0
        self.elmo_labmda = 0.001
        self.elmo_dim = 300

        ################################################################################
        # CNNLayer RNNLayer
        self.use_rnn = False

        if self.corpus_splitting == 1:
            self.embed_dropout = 0.4
        elif self.corpus_splitting == 2:
            self.embed_dropout = 0.4
        elif self.corpus_splitting == 3:
            self.embed_dropout = 0.4
            
        self.cnn_dim = self.wordvec_dim

        if self.need_char or self.need_sub:
            self.cnn_dim += self.char_hid_dim
        if self.need_elmo:
            self.cnn_dim += self.elmo_dim

        if self.corpus_splitting == 1:
            self.cnn_layer_num = 5
        elif self.corpus_splitting == 2:
            self.cnn_layer_num = 4
        elif self.corpus_splitting == 3:
            self.cnn_layer_num = 5

        if self.corpus_splitting == 1:
            self.cnn_kernal_size = [5, 5, 5, 5, 5]
        elif self.corpus_splitting == 2:
            self.cnn_kernal_size = [5, 5, 5, 5]
        elif self.corpus_splitting == 3:
            self.cnn_kernal_size = [3, 3, 3, 3, 3]

        if self.corpus_splitting == 1:
            self.cnn_dropout = 0.4
        elif self.corpus_splitting == 2:
            self.cnn_dropout = 0.4
        elif self.corpus_splitting == 3:
            self.cnn_dropout = 0.4

        self.attned_dim = self.cnn_dim * self.attn_topk
        self.pair_rep_dim = self.attned_dim * 2 * self.cnn_layer_num

        ################################################################################
        # Classifier
        self.clf_class_num = self.four_or_eleven

        if self.corpus_splitting == 1:
            self.clf_fc_num = 0
            self.clf_fc_dim = 2048
        elif self.corpus_splitting == 2:
            self.clf_fc_num = 0
        elif self.corpus_splitting == 3:
            self.clf_fc_num = 0
        
        if self.corpus_splitting == 1:
            self.clf_dropout = 0.3
        elif self.corpus_splitting == 2:
            self.clf_dropout = 0.3
        elif self.corpus_splitting == 3:
            self.clf_dropout = 0.3
        
        if self.corpus_splitting == 1:
            self.conn_num = 94
        elif self.corpus_splitting == 2:
            self.conn_num = 92
        elif self.corpus_splitting == 3:
            self.conn_num = 93
        
        ################################################################################
        self.seed = 666
        self.batch_size = 128
        self.shuffle = True

        if self.corpus_splitting == 1:
            self.lr = 0.001
        elif self.corpus_splitting == 2:
            self.lr = 0.001
        elif self.corpus_splitting == 3:
            self.lr = 0.001
        
        self.l2_penalty = 0
        self.grad_clip = 1
        self.epochs = 10000

        self.is_mttrain = True
        self.lambda1 = 1

        self.logdir = './res/' + datetime.now().strftime('%B%d-%H:%M:%S')

In [4]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import torch.utils.model_zoo as model_zoo

from allennlp.modules.elmo import Elmo

class Atten(nn.Module):
    def __init__(self, conf):
        super(Atten, self).__init__()
        self.conf = conf
        self.w = nn.Conv1d(self.conf.cnn_dim, self.conf.cnn_dim, 1)
        self.temper = np.power(self.conf.cnn_dim, 0.5)
        self.dropout = nn.Dropout(self.conf.attn_dropout)
        self.softmax = nn.Softmax(-1)

    def forward(self, q, v):
        q_ = self.w(q.transpose(1, 2)).transpose(1, 2)
        attn = torch.bmm(q_, v.transpose(1, 2)) / self.temper
        vr = torch.bmm(self.dropout(self.softmax(attn)), v)
        qr = torch.bmm(self.dropout(self.softmax(attn.transpose(1, 2))), q)
        vr = torch.topk(vr, k=self.conf.attn_topk, dim=1)[0]
        vr = vr.view(vr.size(0), -1)
        qr = torch.topk(qr, k=self.conf.attn_topk, dim=1)[0]
        qr = qr.view(qr.size(0), -1)
        return qr, vr, attn

class Highway(nn.Module):
    def __init__(self, size):
        super(Highway, self).__init__()
        self.highway_linear = nn.Linear(size, size)
        self.gate_linear = nn.Linear(size, size)
        self.nonlinear = nn.ReLU()

    def forward(self, input):
        gate = F.sigmoid(self.gate_linear(input))
        m = self.nonlinear(self.highway_linear(input))
        return gate * m + (1 - gate) * input

class RNNLayer(nn.Module):
    def __init__(self, in_dim, conf):
        super(RNNLayer, self).__init__()
        self.conf = conf
        self.rnn = nn.GRU(in_dim, in_dim, num_layers=1,
                            dropout=self.conf.cnn_dropout, bidirectional=True)
        self.conv = nn.Conv1d(in_dim*2, in_dim, 1)
        nn.init.xavier_uniform(self.conv.weight)
        self.conv.bias.data.fill_(0)
        self.dropout = nn.Dropout(self.conf.cnn_dropout)

    def forward(self, input, length, hidden=None):
        lens, indices = torch.sort(length, 0, True)
        maxlen = lens[0]
        outputs, hidden_t = self.rnn(pack(input[indices], lens.tolist(), batch_first=True), hidden)
        outputs = unpack(outputs, batch_first=True)[0]
        _, _indices = torch.sort(indices, 0)
        outputs = outputs[_indices]
        size = outputs.size()
        outputs = F.pad(outputs.unsqueeze(0), (0,0,0,self.conf.max_sent_len-maxlen)).view(size[0],-1,size[2])
        outputs = self.conv(self.dropout(outputs.transpose(1, 2))).transpose(1, 2)
        return outputs + input

class CNNLayer(nn.Module):
    def __init__(self, conf, in_dim, k, res=True):
        super(CNNLayer, self).__init__()
        self.conf = conf
        self.res = res
        self.conv = nn.Conv1d(in_dim, in_dim*2, k, stride=1, padding=k//2)
        nn.init.xavier_uniform(self.conv.weight)
        self.conv.bias.data.fill_(0)
        self.dropout = nn.Dropout(self.conf.cnn_dropout)

    def forward(self, input):
        output = self.dropout(input.transpose(1, 2))
        tmp = self.conv(output)
        if tmp.size(2) > output.size(2):
            output = tmp[:, :, 1:]
        else:
            output = tmp
        output = output.transpose(1, 2)
        a, b = torch.chunk(output, 2, dim=2)
        output = a * nn.functional.sigmoid(b)
        if self.res:
            output = output + input
        return output

class CharLayer(nn.Module):
    def __init__(self, char_table, conf):
        super(CharLayer, self).__init__()
        self.conf = conf
        lookup, length = char_table
        self.char_embed = nn.Embedding(self.conf.char_num, self.conf.char_embed_dim, padding_idx=self.conf.char_padding_idx)
        self.lookup = nn.Embedding(lookup.size(0), lookup.size(1))
        self.lookup.weight.data.copy_(lookup)
        self.lookup.weight.requires_grad = False
        self.convs = nn.ModuleList()
        for i in range(self.conf.char_filter_num):
            self.convs.append(nn.Conv1d(
                self.conf.char_embed_dim, self.conf.char_enc_dim, self.conf.char_filter_dim[i],
                stride=1, padding=self.conf.char_filter_dim[i]//2
            ))
            nn.init.xavier_uniform(self.convs[i].weight)
        self.nonlinear = nn.Tanh()
        self.mask = nn.Embedding(lookup.size(0), self.conf.char_hid_dim)
        self.mask.weight.data.fill_(1)
        self.mask.weight.data[0].fill_(0)
        self.mask.weight.data[1].fill_(0)
        self.mask.weight.requires_grad = False
        self.highway = Highway(self.conf.char_hid_dim)
        del lookup
        del length

    def forward(self, input):
        charseq = self.lookup(input).long().view(input.size(0)*input.size(1), -1)
        charseq = self.char_embed(charseq).transpose(1, 2)
        conv_out = []
        for i in range(self.conf.char_filter_num):
            tmp = self.nonlinear(self.convs[i](charseq))
            if tmp.size(2) > charseq.size(2):
                tmp = tmp[:, :, 1:]
            tmp = torch.topk(tmp, k=1)[0]
            conv_out.append(torch.squeeze(tmp, dim=2))
        hid = torch.cat(conv_out, dim=1)
        hid = self.highway(hid)
        hid = hid.view(input.size(0), input.size(1), -1)
        mask = self.mask(input)
        hid = hid * mask
        return hid

class ElmoLayer(nn.Module):
    def __init__(self, char_table, conf):
        super(ElmoLayer, self).__init__()
        self.conf = conf
        lookup, length = char_table
        self.lookup = nn.Embedding(lookup.size(0), lookup.size(1))
        self.lookup.weight.data.copy_(lookup)
        self.lookup.weight.requires_grad = False
        self.elmo = Elmo(
            os.path.expanduser(self.conf.elmo_options), os.path.expanduser(self.conf.elmo_weights),
            num_output_representations=2, do_layer_norm=False, dropout=self.conf.embed_dropout
        )
        for p in self.elmo.parameters():
            p.requires_grad = False
        self.w = nn.Parameter(torch.Tensor([0.5, 0.5]))
        self.gamma = nn.Parameter(torch.ones(1))
        self.conv = nn.Conv1d(1024, self.conf.elmo_dim, 1)
        nn.init.xavier_uniform(self.conv.weight)
        self.conv.bias.data.fill_(0)

    def forward(self, input):
        charseq = self.lookup(input).long()
        res = self.elmo(charseq)['elmo_representations']
        w = F.softmax(self.w, dim=0)
        res = self.gamma * (w[0] * res[0] + w[1] * res[1])
        res = self.conv(res.transpose(1, 2)).transpose(1, 2)
        return res

class ArgEncoder(nn.Module):
    def __init__(self, conf, we_tensor, char_table=None, sub_table=None, use_cuda=False, attnvis=False):
        super(ArgEncoder, self).__init__()
        self.conf = conf
        self.attnvis = attnvis
        if self.conf.use_rnn:
            self.usecuda = use_cuda
        self.embed = nn.Embedding(we_tensor.size(0), we_tensor.size(1))
        self.embed.weight.data.copy_(we_tensor)
        self.embed.weight.requires_grad = False
        if self.conf.need_char:
            self.charenc = CharLayer(char_table, self.conf)
        if self.conf.need_sub:
            self.charenc = CharLayer(sub_table, self.conf)
        if self.conf.need_elmo:
            self.elmo = ElmoLayer(char_table, self.conf)
        self.dropout = nn.Dropout(self.conf.embed_dropout)

        self.block1 = nn.ModuleList()
        self.block2 = nn.ModuleList()
        self.attn = Atten(self.conf)
        for i in range(self.conf.cnn_layer_num):
            if self.conf.use_rnn:
                self.block1.append(RNNLayer(self.conf.cnn_dim, self.conf))
                self.block2.append(RNNLayer(self.conf.cnn_dim, self.conf))
            else:
                self.block1.append(CNNLayer(self.conf, self.conf.cnn_dim, self.conf.cnn_kernal_size[i]))
                self.block2.append(CNNLayer(self.conf, self.conf.cnn_dim, self.conf.cnn_kernal_size[i]))
    
    def forward(self, a1, a2):
        if self.conf.use_rnn:
            len1 = torch.LongTensor([torch.max(a1[i,:].data.nonzero())+1 for i in range(a1.size(0))])
            len2 = torch.LongTensor([torch.max(a2[i,:].data.nonzero())+1 for i in range(a2.size(0))])
            if self.usecuda:
                len1 = len1.cuda()
                len2 = len2.cuda()
        arg1repr = self.embed(a1)
        arg2repr = self.embed(a2)
        if self.conf.need_char or self.conf.need_sub:
            char1 = self.charenc(a1)
            char2 = self.charenc(a2)
            arg1repr = torch.cat((arg1repr, char1), dim=2)
            arg2repr = torch.cat((arg2repr, char2), dim=2)
        if self.conf.need_elmo:
            arg1repr = torch.cat((arg1repr, self.elmo(a1)), dim=2)
            arg2repr = torch.cat((arg2repr, self.elmo(a2)), dim=2)
        arg1repr = self.dropout(arg1repr)
        arg2repr = self.dropout(arg2repr)
        outputs = []
        attns = []
        for i in range(self.conf.cnn_layer_num):
            if self.conf.use_rnn:
                arg1repr = self.block1[i](arg1repr, len1)
                arg2repr = self.block2[i](arg2repr, len2)
            else:
                arg1repr = self.block1[i](arg1repr)
                arg2repr = self.block2[i](arg2repr)
            outputc1, outputc2, attnw = self.attn(arg1repr, arg2repr)
            outputs.append(outputc1)
            outputs.append(outputc2)
            attns.append(attnw)
        if self.attnvis:
            return torch.cat(outputs, 1), attns
        else:
            return torch.cat(outputs, 1)

class Classifier(nn.Module):
    def __init__(self, nclass, conf):
        super(Classifier, self).__init__()
        self.conf = conf
        self.dropout = nn.Dropout(self.conf.clf_dropout)
        self.fc = nn.ModuleList()
        if self.conf.clf_fc_num > 0:
            self.fc.append(nn.Linear(self.conf.pair_rep_dim, self.conf.clf_fc_dim))
            for i in range(self.conf.clf_fc_num - 1):
                self.fc.append(nn.Linear(self.conf.clf_fc_dim, self.conf.clf_fc_dim))
            lastfcdim = self.conf.clf_fc_dim
        else:
            lastfcdim = self.conf.pair_rep_dim
        self.lastfc = nn.Linear(lastfcdim, nclass)
        self.nonlinear = nn.Tanh()
        self._init_weight()

    def _init_weight(self):
        for i in range(self.conf.clf_fc_num):
            self.fc[i].bias.data.fill_(0)
            nn.init.uniform(self.fc[i].weight, -0.01, 0.01)
        self.lastfc.bias.data.fill_(0)
        nn.init.uniform(self.lastfc.weight, -0.01, 0.01)

    def forward(self, input):
        output = input
        for i in range(self.conf.clf_fc_num):
            output = self.nonlinear(self.dropout(self.fc[i](output)))
        output = self.lastfc(self.dropout(output))
        return output

class IDRCModel(nn.Module):
    def __init__(self, conf, we_tensor, char_table, sub_table, use_cuda):
        super(IDRCModel, self).__init__()
        self.enc = ArgEncoder(conf, we_tensor, char_table, sub_table, use_cuda)
        self.clf = Classifier(conf.clf_class_num, conf)

    def forward(self, arg1, arg2):
        return self.clf(self.enc(arg1, arg2))

def test():
    conf = Config()
    usecuda = True
    we = torch.load('./data/processed/ji/we.pkl')
    char_table = None
    sub_table = None
    if conf.need_char or conf.need_elmo:
        char_table = torch.load('./data/processed/ji/char_table.pkl')
    if conf.need_sub:
        sub_table = torch.load('./data/processed/ji/sub_table.pkl')
    model = IDRCModel(conf, we, char_table, sub_table, usecuda)
    if usecuda:
        model.cuda()
    d = Data(usecuda, conf)
    for a1, a2, sense, conn in d.train_loader:
        if usecuda:
            a1, a2 = a1.cuda(), a2.cuda()
        a1, a2 = Variable(a1), Variable(a2)
        break
    model.eval()
    out = model(a1, a2)
    print(out)

if __name__ == '__main__':
    test()

FileNotFoundError: [Errno 2] No such file or directory: './data/processed/ji/we.pkl'